In [9]:
from git import Repo
import re

url = 'https://github.com/wbkd/react-flow'
url = 'https://github.com/hwchase17/langchain'
# Removing 'https://' or 'http://' prefix
url_formatted_1 = re.sub(r'https?://', '', url)

# Keeping only alphanumeric characters and '-'
url_formatted_2 = re.sub(r'[^a-zA-Z0-9]', '-', url_formatted_1)
index_name = 'pinecone-hackathon-test'

!rm -rf ./clone
repo = Repo.clone_from(url, './clone')
branch = repo.head.reference

In [10]:
from langchain.document_loaders import GitLoader

loader = GitLoader(
    repo_path="./clone", 
    branch=branch, 
    file_filter=lambda x: x.endswith(".js") or x.endswith(".jsx") or x.endswith(".ts") or x.endswith(".tsx") or x.endswith(".json") or x.endswith(".md") or x.endswith(".py"))
data = loader.load()

len(data)

# print(data[0])

1241

In [11]:
# data

In [12]:
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
from os import environ

environ.get("PINECONE_ENVIRONMENT")

'northamerica-northeast1-gcp'

In [14]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [15]:
import pinecone
from langchain.vectorstores import Pinecone
from os import environ

pinecone.init(
    # api_key=environ.get("PINECONE_API_KEY"), 
    # environment=environ.get("PINECONE_ENVIRONMENT")
)
# index_name = 'pinecone-hackathon-test'

if index_name not in pinecone.list_indexes():
    index= pinecone.create_index(
        name=index_name, 
        metric='cosine', 
        shards=1, 
        dimension=1536
    )
index = pinecone.Index(index_name=index_name)

index = Pinecone.from_documents(
    data, 
    embeddings, 
    index_name=index_name, 
    batch_size=10,
    namespace = url_formatted_2
)
# for x in range(len(data)):
#     print("adding: " + str(x))
#     if len(data[x].page_content) < 100000:
#         index = Pinecone.from_documents([data[x]], embeddings, index_name=index_name)
    
# pinecone.add_documents(data, index_name=index_name)
# index
index

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'date': 'Wed, 21 Jun 2023 01:00:15 GMT', 'x-envoy-upstream-service-time': '1', 'content-length': '115', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"metadata size is 88093 bytes, which exceeds the limit of 40960 bytes per vector","details":[]}


In [ ]:
import pinecone 
pinecone.describe_index(index_name)

# index.describe_index_stats()

IndexDescription(name='pinecone-hackathon-test', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')